In [1]:
import pandas as pd
import threading
import datetime as dt
import config as cfg
import sounddevice as sd
from scipy.io.wavfile import write
import os

In [2]:
def verificar_y_crear_directorio(ruta_directorio):

    # Verificar si el directorio ya existe
    if not os.path.exists(ruta_directorio):
        try:
            # Intentar crear el directorio si no existe
            os.makedirs(ruta_directorio)
            print(f"Directorio '{ruta_directorio}' creado exitosamente.")
        except OSError as error:
            print(f"Error al crear el directorio '{ruta_directorio}': {error}")
    else:
        print(f"El directorio '{ruta_directorio}' ya existe.")

verificar_y_crear_directorio("recordings")
verificar_y_crear_directorio("inferences")

El directorio 'recordings' ya existe.
El directorio 'inferences' ya existe.


In [3]:
def grabar_audio(nombre_archivo, duracion=10):
    
    print(f"Grabando audio por {duracion} segundos...")

    # Configurar la grabación
    sr = cfg.SAMPLE_RATE 
    recording = sd.rec(int(duracion * sr), samplerate=sr, channels=2)
    sd.wait()

    # Guardar la grabación en un archivo WAV
    nombre_completo = f"recordings/{nombre_archivo}.wav"
    write(nombre_completo, sr, recording)

    print(f"Audio guardado como {nombre_completo}")

def borrar_audio(nombre_archivo):
    
    try:
        os.remove(f"recordings/{nombre_archivo}.wav")
        print(f"Archivo {nombre_archivo}.wav eliminado.")
    except FileNotFoundError:
        print(f"Archivo {nombre_archivo}.wav no encontrado.")

def borrar_archivos_si_no_cumplen_condicion(input_file, columna, valor_umbral):
    
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"El archivo {input_file} no fue encontrado.")
        return False  

    # Verificar si las columnas especificadas existen en el DataFrame
    if columna not in df.columns or 'Filename' not in df.columns:
        print("Una o ambas columnas especificadas no existen en el archivo CSV.")
        return False

    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        archivo = row['Filename']
        condicion = row[columna]

        # Verificar la condición
        if condicion < valor_umbral:
            # Eliminar el archivo si no cumple la condición
            if os.path.exists(archivo):
                os.remove(archivo)
                print(f"Se ha eliminado el archivo '{archivo}' porque '{columna}' < {valor_umbral}.")

    return True

In [4]:
# CSV output file
header = "Filename,Start (s),End (s),Scientific name,Common name,Confidence\n"
resultsPath = "inferences/results.csv"

if os.path.dirname(resultsPath):
    os.makedirs(os.path.dirname(resultsPath), exist_ok=True)

with open(resultsPath, "w", encoding="utf-8") as rfile:
    rfile.write(header)

In [12]:

current_date = dt.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

nombre_archivo = str(current_date)    
duracion_grabacion = 10

grabar_audio(nombre_archivo, duracion_grabacion)

def inference():
    comando_a_ejecutar = f"python3 analyze.py --i recordings/{nombre_archivo}.wav --o {resultsPath} --locale es --rtype csv --min_conf 0.5"
    os.system(comando_a_ejecutar) 

    borrar_archivos_si_no_cumplen_condicion(resultsPath, 'Confidence', 0.7)

p = threading.Thread(target=inference)
p.start()

Grabando audio por 10 segundos...
Audio guardado como recordings/16-04-2024_11-14-14.wav


Species list contains 6522 species
Analyzing recordings/16-04-2024_11-14-14.wav


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Finished recordings/16-04-2024_11-14-14.wav in 2.30 seconds


In [15]:
df = pd.read_csv(resultsPath)
df

,Filename,Start (s),End (s),Scientific name,Common name,Confidence
0,16-04-2024_11-32-43.wav,9.0,12.0,Glaucidium gnoma,Mochuelo Californiano,0.6018
1,16-04-2024_11-32-53.wav,0.0,3.0,Human whistle,Human whistle,0.5461
2,16-04-2024_11-32-53.wav,3.0,6.0,Human whistle,Human whistle,0.5797
3,16-04-2024_11-33-03.wav,0.0,3.0,Nyctibius griseus,Nictibio Urutaú,0.9017
4,16-04-2024_11-33-03.wav,3.0,6.0,Cuculus micropterus,Cuco Alicorto,0.7678
5,16-04-2024_11-33-03.wav,6.0,9.0,Human whistle,Human whistle,0.7864
6,16-04-2024_11-33-23.wav,0.0,3.0,Meleagris gallopavo,Guajolote Gallipavo,0.7432
7,16-04-2024_11-33-23.wav,3.0,6.0,Meleagris gallopavo,Guajolote Gallipavo,0.7629
